# 🍳 Recipe Generator

**Goal:** Input 3 ingredients (e.g., "Eggs, Tomato, Cheese") and generate a cooking instruction paragraph.

**Tech:** Text generation with a specific prompt format.

**Model Used:** flax-community/t5-recipe-generation (Chef Transformer) - T5-based, best quality, trained on RecipeNLG Dataset

(Chef Transformer is a T5-based encoder–decoder model (~220M params) trained on 2M+ recipes from the RecipeNLG dataset. It excels at structured, realistic recipe generation, producing proper titles, ingredients, steps, and accurate cooking measurements using rich culinary vocabulary.)

In [2]:
import torch
from transformers import pipeline
import warnings

warnings.filterwarnings('ignore')

if torch.backends.mps.is_available():
    device = "mps"
    print("Using Apple M4 GPU (MPS)")
elif torch.cuda.is_available():
    device = 0
    print("Using CUDA GPU")
else:
    device = -1
    print("Using CPU")

Using Apple M4 GPU (MPS)


In [ ]:
# Loading Chef Transformer
print("Loading Chef Transformer via Pipeline...")
recipe_pipe = pipeline(
    "text2text-generation", 
    model="flax-community/t5-recipe-generation",
    device=device
)
print("Pipeline loaded successfully!")

Loading Chef Transformer via Pipeline...
Pipeline loaded successfully!


In [10]:
# Recipe Generation
def generate_and_format_recipe(ingredients, num_variations=1):
    """
    Uses the pipeline to generate recipes and applies formatting for readability.
    """
    input_text = f"items: {', '.join(ingredients)}"
    
    # Generate using the pipeline
    results = recipe_pipe(
        input_text,
        max_length=512,
        min_length=64,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=60,
        top_p=0.95,
        num_return_sequences=num_variations
    )
    
    formatted_outputs = []
    
    for res in results:
        raw_text = res['generated_text']
        
        # Post-processing: Chef Transformer uses <sep> for lists and <section> for breaks
        # We clean these into a readable structure
        sections = raw_text.split("<section>")
        
        title = sections[0].replace("title:", "").strip() if len(sections) > 0 else "Untitled Recipe"
        
        # Clean ingredients and directions
        body = ""
        for section in sections[1:]:
            section = section.replace("ingredients:", "\n🥘 Ingredients:").replace("directions:", "\n👨‍🍳 Directions:")
            section = section.replace("<sep>", "\n  • ")
            body += section + "\n"

        recipe_card = f"📖 RECIPE\n" + "="*40 + f"\n🍽️  Title: {title}\n{body}" + "="*40
        formatted_outputs.append(recipe_card)
        
    return formatted_outputs

In [12]:
# Interactive Interface
def run_generator():
    print("\n🍽️  Bullet-Point Recipe Generator")
    print("="*45)
    
    user_input = input("Enter ingredients separated by commas: ")
    ingredients = [i.strip() for i in user_input.split(",")] if user_input else ["chicken breast", "lemon", "garlic"]
    
    print(f"\n✨ Creating your recipe for: {', '.join(ingredients)}...\n")
    
    recipes = generate_and_format_recipe(ingredients)
    
    for recipe in recipes:
        print(recipe)

# Run the application
run_generator()


🍽️  Bullet-Point Recipe Generator

✨ Creating your recipe for: penne, cheese, tomato...

📖 RECIPE
🍽️  Title: penne with cheese and sun dried tomatoes ingredients: 1 pkg. 16 oz. refrigerated pizza crust dough 1 cup kraft 100 parmesan grated cheese 1 tbsp. kraft sun dried tomato oregano dressing directions: heat oven to 425 degrees f. unroll pizza dough onto baking sheet sprayed with cooking spray press into 15x10x1 inch rectangle. top with cheese, tomatoes and dressing. bake 15 min. or until golden brown.
